In [4]:
# Import libraries
import math
import pandas as pd
import numpy as np
import numpy_financial as npf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from pandas.core.common import random_state
from sklearn.linear_model import LinearRegression

# Module 57.1: The Term Structure of Interest Rates: Spot, Par, and Forward Curves

<hr>

Spot rate - no arbitrage principle that a 3-year spot rate should be the same as investing for one year in three successive periods


The following [Finding a Forward Rate](https://www.kaplanlearn.com/education/dashboard/index/6e0d53e6124eca676dff518e031311f3/course/89414904/node/cfaL1_topic_00439_3.dita#cfaL1_topic_00439_3/fig01_cfaL1_topic_00439_3) illustrates the problem.


<center><i>Finding a Forward Rate</i></center>


<img src="https://github.com/PachaTech/CFA-Level-1/blob/main/Fixed%20Income/Module%2057/pics/forward.jpeg?raw=true"></img>

### LOS 57.a: Define spot rates and the spot curve, and calculate the price of a bond using spot rates.

The yield to maturity of a bond is calculated as if the discount rate for every bond cash flow is the same. In reality, discount rates vary according to the time period in which a future cash flow is made. These discount rates for a single payment to be received in the future are called **spot rates** and can be observed by calculating the discount rates for zero-coupon bonds (hence, spot rates are sometimes referred to as *zero-coupon rates* or simply *zero rates*).

To price a bond with spot rates, we sum the present values of the bond's payments, each discounted at the spot rate for the number of periods before it will be paid. The general equation for calculating a bond's value using spot rates $(S_i)$ is as follows:

$PV= \Large\frac{\text{CPN_1}}{1+S_1} + \frac{CPN_2}{(1+S_2)^2} + ... + \frac{CPN_N + FV_N}{(1+S_N)^N}$


<hr>

**EXAMPLE: VALUING A BOND USING SPOT RATES**

Given the following spot rates, calculate the value of a 3-year, 5% annual coupon bond.

Spot rates:

1-year: 3%

2-year: 4%

3-year: 5%

**Answer:**

$\Large\frac{5}{(1.03)^1}+\frac{5}{(1.04)^2}+\frac{105}{(1.05)^3} = 100.180$

* 3-year, 5% Annual-Pay
* \\$100 Par 

<table>
<thead>
  <tr>
    <th>Time</th>
    <th>Cashflow</th>
    <th>Discount Factor</th>
    <th>Present Value</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>1</td>
    <td>5</td>
    <td>1 / 1.03^1</td>
    <td>4.854</td>
  </tr>
  <tr>
    <td>2</td>
    <td>5</td>
    <td>1 / 1.04^2</td>
    <td>4.623</td>
  </tr>
  <tr>
    <td>3</td>
    <td>105</td>
    <td>1 / 1.05^3</td>
    <td>90.703</td>
  </tr>
  <tr>
    <td></td>
    <td></td>
    <td></td>
    <td>100.180</td>
  </tr>
</tbody>
</table>



In [86]:
# EXAMPLE: VALUING A BOND USING SPOT RATES
SR = (5 / (1.03)**1) + (5 / (1.04)**2) + (105 / (1.05)**3)
print(round(SR,3))

100.18


This price, calculated using spot rates, is sometimes called the no-arbitrage price of a bond because if a bond is priced differently, there will be a profit opportunity from arbitrage among bonds.

Because the bond value is slightly greater than its par value, we know its YTM is slightly less than its coupon rate of 5%. Using the price of 100.180, we can calculate the YTM for this bond as follows:

    N = 3; PMT = 5; FV = 100; PV = –100.180; CPT → I/Y = 4.93%

> **Professor's Note**\
It's useful to think of a spot rate as a return offered by a single cash flow occurring at a certain time in the future. A weighted average of these spot rates gives us an idea of what the yield is likely to be. For the bond in the previous example, we have the first coupon returning 3%, the second coupon returning 4%, and the third coupon and par payment returning 5%. Given that most of the cash flows occur at Time 3 when the par payment is made, the average is going to be heavily weighted to 5%. This means we can easily see that the yield must be just below 5%.

The **spot curve** displays spot rates versus maturity for a particular type of bond or issuer (e.g., U.S. Treasury government spot rates). We will construct and use spot curves later in this reading.

In [87]:
# Calculate YTM bond rate of 100 using numpyfinance 
bond_rate = npf.rate(pv=-100.18, nper=3,fv= 100, pmt =5)


# Print PV of the $100 note
print("Bond YTM rate = %",round(bond_rate,6)*100)

Bond YTM rate = % 4.934


### LOS 57.b: Define par and forward rates, and calculate par rates, forward rates from spot rates, spot rates from forward rates, and the price of a bond using forward rates.

#### Par Yields

**Par yields** reflect the coupon rate that a hypothetical bond at each maturity would need to have to be priced at par, given a specific spot curve. Alternatively, they can be viewed as the YTM of a hypothetical par bond at each maturity.

Consider a 3-year annual-pay bond and spot rates for one, two, and three years of S1, S2, and S3. The following equation can be used to calculate the coupon rate, PMT, necessary for the bond to be trading at par:

$\Large\frac{\text{PMT}}{1+S_1} + \frac{PMT_2}{(1+S_2)^2} + ... + \frac{PMT + 100}{(1+S_3)^3} = 100$


With spot rates of 1%, 2%, and 3%, a 3-year annual par bond will have a payment that will satisfy the following:

$\Large\frac{\text{PMT}}{1.01} + \frac{PMT_2}{(1.02)^2} + ... + \frac{PMT + 100}{(1.03)^3} = 100$


In this case, the payment is 2.96 and the par bond coupon rate is 2.96%.

>**Professor's Note**\
If this type of calculation appears on the exam, the best tactic is to plug in the middle answer choice and see if it gives a value of 100. If it results in a value less than 100, the larger choice must be correct. If it produces a value greater than 100, the smaller choice must be correct. Algebraically solving for PMT can be done, but it takes much more time than simply trying the coupon rates in the answers.

#### Forward Rates

A forward rate is a borrowing/lending rate for a loan to be made at some future date. The notation used must identify both the start and length of the lending/borrowing period. The most common convention (and the one used on the Level I CFA exam) is to denote a forward period using two numbers, each followed by a letter indicating the compounding period (y for years, m for months). The first number represents the future period in which the loan begins, and the second denotes the length of the loan. Hence, 2y1y is the rate for a 1-year loan to be made two years from now; 3y2y is the 2-year forward rate three years from now; 1y1y is the rate for a 1-year loan one year from now; and so on.

When linking forward rates to spot rates—say, for example, over the next three years—a key no-arbitrage idea is that borrowing for three years at the 3-year spot rate, or borrowing for one-year periods in three successive years, should have the same cost.

This relation is illustrated as $\large (1 + S_3)^3 = (1 + S_1)(1 + 1y1y)(1 + 2y1y)$.

Thus, $\Large S_3 = [(1 + S_1)(1 + 1y1y)(1 + 2y1y)]^{1/3} − 1$, which is a basic geometric mean periodic return.

More generally, the spot rate earned between now and a future maturity must be equal to the compounded forward rates that apply to each period out to that maturity.

<hr>

**EXAMPLE: COMPUTING SPOT RATES FROM FORWARD RATES**

If the current 1-year spot rate is 2%, the 1-year forward rate one year from today (1y1y) is 3%,

and the 1-year forward rate two years from today (2y1y) is 4%, what is the 3-year spot rate?

In [88]:
# EXAMPLE: COMPUTING SPOT RATES FROM FORWARD RATES

S_3 = (1.02 * 1.03 * 1.04)**(1/3) - 1
print("S_3 is = %", round(S_3,5)*100)

S_3 is = % 2.997


**Answer:**

$\Large S_3 = [(1.02)(1.03)(1.04)]^{1/3} − 1 = 2.997\%$

This can be interpreted to mean that a dollar compounded at 2.997% for three years would produce the same ending value as a dollar that earns compound interest of 2% the first year, 3% the next year, and 4% for the third year.


>**Professor's Note**\
You can get a very good approximation of the 3-year spot rate with the simple average of the forward rates. In the previous example, we calculated 2.997%, and the simple average of the three annual rates is as follows:

$\qquad\large\frac{2+3+4}{3}=3\%$

We can use the same relationships we use to calculate spot rates from forward rates to calculate forward rates from spot rates.

This is our basic relation between forward rates and spot rates (for two periods):

$\qquad(1 + S_2)^2 = (1 + S_1)(1 + 1y1y)$

This again tells us that an investment has the same expected yield (borrowing has the same expected cost) whether we invest (borrow) for two periods at the 2-period spot rate, $S_2$, or for one period at the current 1-year rate, $S_1$, and for the next period at the forward rate, $1y1y$. Given two of these rates, we can solve for the othe

<hr>

**EXAMPLE: COMPUTING A FORWARD RATE FROM SPOT RATES**

The 2-period spot rate, $S_2$, is 8%, and the 1-period spot rate, $S_1$, is 4%. Calculate the forward rate for one period, one period from now, $1y1y$.

In [89]:
# EXAMPLE: COMPUTING A FORWARD RATE FROM SPOT RATES
S_1y1y = (1.08)**2 / 1.04 - 1
print(round(S_1y1y,5)*100)

12.154


**Answer:**

From our original equality, $\large (1 + S_2)^2 = (1 + S_1)(1 + 1y1y)$, we can get:


$\Large\frac{\text{1+S_2}^2}{1+S_1} = (1+1y1y)$

Or, because we know that both choices have the same payoff in two years:

$(1.08)_2 = (1.04)(1 + 1y1y)$

$\qquad(1+1y1y)= \large\frac{{(1.08)^2}}{1.04}$

$\qquad(1+1y1y)= \large\frac{(1.08)^2}{1.04} -1 = \frac{1.1664}{1.04}-1 = 12.154\%$

In other words, investors are willing to accept 4.0% on the 1-year bond today (when they could get 8.0% on the 2-year bond today) only because they can get 12.154% on a 1-year bond one year from today. This future rate that can be locked in today is a forward rate.

Similarly, we can back other forward rates out of the spot rates. We know that:

$(1 + S_3)^3 = (1 + S_1)(1 + 1y1y)(1 + 2y1y)$

And that:

$(1 + S_2)^2 = (1 + S_1)(1 + 1y1y)$, so we can write $(1 + S_3)^3 = (1 + S_2)^2(1 + 2y1y)$.

This last equation says that investing for three years at the 3-year spot rate should produce the same ending value as investing for two years at the 2-year spot rate—and then for a third year at 2y1y, the 1-year forward rate, two years from now.

Solving for the forward rate, $2y1y$, we get:

$\qquad\large\frac{(1+S_3)^3}{(1+S_2)^2}−1 = 2y1y$


<hr>

**EXAMPLE: FORWARD RATES FROM SPOT RATES**

Let's extend the previous example to three periods. The current 1-year spot rate is 4.0%, the current 2-year spot rate is 8.0%, and the current 3-year spot rate is 12.0%. Calculate the 1-year forward rate two years from now.

In [90]:
# EXAMPLE: FORWARD RATES FROM SPOT RATES

# (1 + S_3)^3 = (1 + S_2)^2 (1 + 2y1y)

# Substituting values for S_3 and S_2, we have:
# (1.12)^3 = (1.08)^2 * (1 + 2y1y)


# This is so that the 1-year forward rate two years from now is
f_rate_2y1y = (1 + 0.12)**3 / (1 + 0.08)**2 - 1
print("The 1-year forward rate, two years from now is = %", round(f_rate_2y1y,5)*100)


# We can check our results by calculating:

S_3 = (1.04  * 1.12154 * 1.2045)**(1/3) - 1
print("S_3 is = %", round(S_3,2)*100)

The 1-year forward rate, two years from now is = % 20.45
S_3 is = % 12.0


This may all seem a bit complicated, but the basic relation, that borrowing for successive periods at 1-period rates should have the same cost as borrowing at multiperiod spot rates, can be summed up as follows:

$(1 + S_2)^2 = (1 + S_1)(1 + 1y1y)$ for <u>two periods</u>, and $(1 + S_3)^3 = (1 + S_2)^2(1 + 2y1y)$ for <u>three periods</u>

>**Professor's Note**\
Simple averages also give decent approximations for calculating forward rates from spot rates (particularly when rates are small). In the preceding example, we had spot rates of 4% for one year and 8% for two years. Two years at 8% is 16%, so if the first-year rate is 4%, the second-year forward rate is close to 16 – 4 = 12% (actual is 12.154). Given a 2-year spot rate of 8% and a 3-year spot rate of 12%, we could approximate the 1-year forward rate from Time 2 to Time 3 as (3 × 12) – (2 × 8) = 20. That may be close enough (actual is 20.45) to answer a multiple-choice question—and, in any case, it serves as a good check to make sure the exact rate you calculate is reasonable.

We can also calculate implied forward rates for loans for more than one period. Given spot rates (1-year = 5%, 2-year = 6%, 3-year = 7%, and 4-year = 8%), we can calculate 2y2y.


The implied forward rate on a 2-year loan two years from now, 2y2y, is as follows:

$\qquad\Biggr[\large\frac{(1+S_4)^4}{(1+S_2)^2}\Biggr]^{1/2} - 1 = \Bigg(\large\frac{1.08^4}{1.06^2}\Bigg)^{1/2} -1 = 10.04\%$

>**Professor's Note**\
The approximation works for multiperiod forward rates as well. 
The difference between four years at 8% (= 32%) and two years at 6% (= 12%) is 20%. Because that difference is for two years, we divide by two to get an annual rate of 10%, $\frac{(4×8−6×2)}{2}=10$, which is very close to the exact solution of 10.04%.

<hr>

**EXAMPLE: COMPUTING A BOND VALUE USING FORWARD RATES**

The current 1-year rate, S1, is 4%, the 1-year forward rate for lending from time = 1 to time = 2 is 1y1y = 5%, and the 1-year forward rate for lending from time = 2 to time = 3 is 2y1y = 6%. Value a 3-year annual-pay bond with a 5% coupon and a par value of $1,000.

In [91]:
# EXAMPLE: COMPUTING A BOND VALUE USING FORWARD RATES

# value = 50/(1+S1) + 50/(1+S1)(1+1y1y) + 1050/(1+S1)(1+1y1y)(1+2y1y)

bond_value = 50/(1.04) + 50/(1.04*1.05) + 1050/(1.04*1.05*1.06)
print("Bond Value computed using forward rates is = $", round(bond_value,2))

Bond Value computed using forward rates is = $ 1000.98


>**Professor's Note**\
\
If you think this looks a little like valuing a bond using spot rates, as we did\
for arbitrage-free valuation, you are correct. The discount factors are 
equivalent to spot rate discount factors.\
\
If we have a semiannual coupon bond, the calculation methods are the \
same, but we would use the semiannual discount rate rather than the \
annualized rate—and the number of periods would be the number of \
semiannual periods.



### LOS 57.c: Compare the spot curve, par curve, and forward curve.

The spot rate yield curve (spot curve) for U.S. Treasury bonds, also referred to as the zero curve or the strip curve (because zero-coupon U.S. Treasury bonds are also called stripped Treasuries), is a plot of spot rates versus maturity.

Usually, a spot curve is upward sloping, with higher spot rates for longer maturities (referred to as a normal yield curve), reflecting investor demand for higher returns over longer time frames. When spot rates are lower for longer-dated maturities, the spot curve is downward-sloping and is said to be inverted. Spot rates are usually quoted on a semiannual bond basis, so they are directly comparable to YTMs quoted for coupon government bonds.

A **yield curve for coupon bonds** shows the YTMs for a similar type of actively traded coupon bonds at various maturities (e.g., U.S. Treasury bonds). Yields are calculated for several available maturities, and yields for bonds with maturities between these are estimated by linear interpolation. Yields are usually expressed on a semiannual bond basis.

A practical issue with constructing yield curves directly from traded coupon bond prices is that distortions in yields can occur due to illiquidity and taxation differences between interest income and gains and losses from buying bonds at a discount or premium. To avoid the illiquidity issue, "on-the-run" (most recently issued) bond yields are used; however, there may not be enough on-the-run securities to construct a meaningful curve, and the ones that do exist may trade at a premium or discount to par, causing taxation distortions.

To avoid these practical issues when constructing a coupon bond yield curve, a **par bond yield curve**, or *par curve*, can be constructed from spot curves. As we discussed earlier, par yields are hypothetical yields of bonds that would trade at par for a specific maturity. A par yield curve displays the yields of par bonds versus maturity.

A **forward yield curve** shows forward rates for bonds or money market securities for annual periods in the future. Typically, the forward curve would show the yields of 1-year securities for each future year, quoted on a semiannual bond basis.

As we have seen, the spot rate for a given maturity is a geometric average of the forward rates that apply to each period between now and that maturity. Therefore, when the forward curve is upward sloping, the spot curve is also upward sloping, but less so. We also saw earlier how a par yield at a certain maturity is a weighted average of the spot rates that apply to the individual cash flows of the bond (most heavily weighted toward the longest-dated spot rate when par payment takes place). Hence, par yields will also be upward sloping, and very close to spot rates (but slightly below them) in a normal (upward-sloping) forward curve environment.

>**Professor's Note**\
\
It might be helpful, once again, to approximate using simple averages to 
get a clear picture. Assume the 1-year spot rate is 1% and the 1y1y forward rate is 3%. 
Note that we have rising periodic rates here (the forward curve is rising with maturity). 
The annualized 2-year spot is approximated as (1% + 3%) / 2 = 2%, so we can see the spot curve 
is also rising with maturity, but at a slower pace than forward rates. 
\
\
The yield of a 1-year annual coupon bond would be 1% because there would only be one cash flow at maturity for this bond, and it would therefore earn the 1-year spot rate. The 2-year yield would be a weighted average of the 1-year spot of 1% and the 2-year spot of 2%, but mostly 2% because that is where most of the 2-year bond's cash flows occur. So, we can see that par yields, too, are rising with maturity—and are close to, but slightly below, spot rates. The key takeaway here is that forward rates drive spot rates, which in turn drive par yields.


Likewise, when the forward curve is downward sloping, the spot curve will also be downward sloping, but less so; and par yields will also be downward sloping, and close to spot rates (but slightly above them). One potential explanation for an inverted yield curve is that interest rates are expected to decrease.

When forward rates are constant, it means that all future periodic rates are the same. This means that spot rates to all maturities will be the same—and therefore, bond yields will be the same for all maturities. We describe this as a flat yield curve environment.

## Reading 57: Key Concepts

<hr>

### LOS 57.a

Spot rates are market discount rates for single payments to be made in the future.

The no-arbitrage price of a bond is calculated using no-arbitrage spot rates as follows:

$\qquad\text{no-arbitrage price} = \large\frac{\text{coupon}}{(1+S_1)} + \frac{\text{coupon}}{(1+S_2)^2} + \ldots + \frac{\text{coupon + principal}}{(1+S_N)^N}$

### LOS 57.b

Spot curves can be used to derive the par yields of hypothetical bonds trading at par across different maturities. It is useful to interpret the par yield of a bond as a weighted average of the spot rates applying to each individual cash flow of the bond.

Forward rates are current lending/borrowing rates for short-term loans to be made in future periods.

A spot rate for a maturity of N periods is the geometric mean of forward rates over the N periods. The same relation can be used to solve for a forward rate given spot rates for two different periods.

To value a bond using forward rates, discount the cash flows at Periods 1 to N by the product of one plus each forward rate for Periods 1 to N, and sum them.

**This is for a 3-year annual-pay bond:**


$\quad\text{price} = \large\frac{\text{coupon}}{(1+S_1)} + \frac{\text{coupon}}{(1+S_1)(1+1y1y)}+\frac{\text{coupon + principal}}{(1+S_1)(1+1y1y)(1+2y1y)} $


### LOS 57.c

The spot curve plots spot rates versus maturity. It can be derived from the prices of instruments offering single payments in the future, such as zero-coupon bonds or stripped Treasury bonds.

The par curve shows the coupon rates for bonds of various maturities that would result in bond prices equal to their par values.

A forward curve is a yield curve composed of forward rates, such as 1-year rates, available at each year over a future period.

In an upward-sloping (normal) yield curve environment, forward rates will be higher than spot rates, which will be higher than par yields. In a downward-sloping (inverted) yield curve environment, forward rates will be lower than spot rates, which will be lower than par yields. In a flat yield curve environment, forward rates will be equal to spot rates, which will be equal to par yields across all maturities.

# Module Quiz 57.1 Examples
<hr>


**Question 1:** 

If spot rates are 3.2\% for one year, 3.4\% for two years, and 3.5\% for three years, the price of a \\$100,000 face value, 3-year, annual-pay bond with a coupon rate of 4\% is closest to:

* **ANSWER:** \\$101,419.28

* **EXPLANATION:**
$$
\text{bond value}=\frac{4,000}{1.032}+\frac{4,000}{(1.034^2)}+\frac{104,000}{(1.035^3)}= $101,419.28
$$
\
(Module 57.1, LOS 57.a)

In [8]:
# Question 1
b_value= 4000/(1.032) + 4000/(1.034**2) +104000/(1.035**3)
print(round(b_value,2))

101419.28


**Question 2:** 

A market rate of discount for a single payment to be made in the future is a:

* **ANSWER:** spot rate.
* **EXPLANATION:** \
    A spot rate is a discount rate for a single future payment. Simple yield is a measure of a bond's yield that accounts for coupon interest and assumes straight-line amortization of a discount or premium. A forward rate is an interest rate for a future period, such as a 3-month rate six months from today.\
(Module 57.1, LOS 57.a)

**Question 3:** 

Which of the following yield curves is *least likely* to consist of observed yields in the market?

* **ANSWER:** Par bond yield curve.
* **EXPLANATION** 
    Par bond yield curves are based on the theoretical yields that would cause bonds at each maturity to be priced at par. Coupon bond yields and forward interest rates can be observed directly from market transactions.\
(Module 57.1, LOS 57.b)

**Question 4:** 

The 4-year spot rate is 9.45%, and the 3-year spot rate is 9.85%. What is the 1-year forward rate three years from today?

* **ANSWER:** \
8.258%
* **EXPLANATION:**

$$(1.0945)^4 =(1.0985)^3 × (1 + 3y1y)$$

$$3y1y= \frac{(1.0945)^4}{(1.0985)^3}−1 = 8.258\%$$

$\text{approximate forward rate} = 4(0.0945) − 3(0.0985) = 0.0825$

(Module 57.1, LOS 57.b)

In [93]:
# Question 4:
# The 4-year spot rate is 9.45%, and the 3-year spot rate is 9.85%. 
# What is the 1-year forward rate three years from today?

fr_3y1y = (1.0945**4)/(1.0985**3) - 1
print(round(fr_3y1y,6))

0.082587


**Question 5:** \
Given the following spot and forward rates:

* Current 1-year spot rate is 5.5%.
* 1-year forward rate one year from today is 7.63%.
* 1-year forward rate two years from today is 12.18%.
* 1-year forward rate three years from today is 15.5%.

The value of a 4-year, 10% annual-pay, $1,000 par value bond is closest to:

* **ANSWER:** \
8.258\%

* **EXPLANATION:** 

$$
1y4y = \frac{100}{1.055}+\frac{100}{(1.055)(1.0763)}+\frac{100}{(1.055)(1.0763)(1.1218)}+\frac{1100}{(1.055)(1.0763)(1.1218)(1.155)} = $1009.03
$$

\
(Module 57.1, LOS 57.b)

In [9]:
## Question 5 Module 57.1 Quiz

# Given the following spot and forward rates:

# Current 1-year spot rate is 5.5%.
# 1-year forward rate one year from today is 7.63%.
# 1-year forward rate two years from today is 12.18%.
# 1-year forward rate three years from today is 15.5%.
# The value of a 4-year, 10% annual-pay, 
# $1,000 par value bond is closest to:

FR_1y4y = (100 / 1.055) + (100 / (1.055 * 1.0763)) + (100/(1.055*1.0763*1.1218)) + (1100/(1.055*1.0763*1.1218*1.155))
print(round(FR_1y4y,2))

1009.03


**Question 6:** 

The 1-year spot rate is 1% and the 1y1y rate is 3%. Which of the following statements is most accurate?

* **ANSWER:** \
    The 2-year spot is just below 2%, and the 2-year par yield is just below the 2-year spot rate.
* **EXPLANATION:** \
    The 2-year spot rate must reflect the periodic rates of 1% in the first period and 3% in the second period—hence, the 2-year spot will be approximately 2%. Given that forward rates are rising with maturity, spot rates will be rising with maturity at a slower pace, and par yields will also be rising, but slightly below spot rates. So, the correct answer is that the 2-year spot is just below 2%, and the 2-year par yield is just below the 2-year spot rate. (Module 57.1, LOS 57.c)

<hr>

# Module 57 Qbank 

<hr>

**QUESTION 1/6:**
The one-year spot rate is 5% and the two-year spot rate is 6.5%. What is the one-year forward rate starting one year from now?

* **ANSWER:**  \
    8.02%
* **EXPLANATION:** \
    The forward rate is computed as follows:
$$
\text{One-year forward rate} = \frac{1.065^2}{1.05} – 1 = 8.02\%
$$
(Module 57.1, LOS 57.b)

In [10]:
# Question 1 of 6
Q1 = ((1.065**2) / 1.05) - 1
print(round(Q1,4)*100)

8.02


**QUESTION 2/6:**

Using the following spot rates, what is the price of a three-year bond with annual coupon payments of 5%?

* One-year rate: 4.78%
* Two-year rate: 5.56%
* Three-year rate: 5.98%

<hr>

* **ANSWER:**  \
    \\$97.47
* **EXPLANATION:** \
    The bond price is computed as follows:
$$
\text{Bond price} = \frac{5}{1.0478} + \frac{5}{1.0556^2} + \frac{105} {1.0598^3} = \$97.47
$$
\
    (Module 57.1, LOS 57.a)

In [11]:
# Question 2 of 6
# EXAMPLE: VALUING A BOND USING SPOT RATES
SR_bond = (5 / (1.03)**1) + (5 / (1.04)**2) + (105 / (1.05)**3)
print(round(SR,2))

NameError: name 'SR' is not defined

**QUESTION 3/6:**

Given that the two-year spot rate is 5.89% and the one-year forward rate one-year from now is 6.05%, assuming annual compounding what is the one year spot rate?

* **ANSWER:**  \
    5.73\%
* **EXPLANATION:** \
    The spot rate is computed as follows:
$$
\text{spot-rate}_{0,1} = \frac{\text{1 + spot rate 0,1}}{\text{1 + forward rate 1,2}^1} - 1 = \frac{(1+0.0589)^2}{(1+0.0605)^1}
-1 = 5.73\%$$
\
    (Module 57.1, LOS 57.a)

In [12]:
# Question 3 of 6
# Given that the two-year spot rate is 5.89% and 
# the one-year forward rate one-year from now is 6.05%, 
# assuming annual compounding what is the one year spot rate?

sr01 = (1.0589**2)/(1.0605) - 1 

print(round(sr01,4))

0.0573


**QUESTION 4/6:**

Given that the 2-year spot rate is 5.76% and the 3-year spot rate is 6.11%, what is the 1-year forward rate starting two years from now?

* **ANSWER:**  \
    6.81\%
* **EXPLANATION:** \
    The spot rate is computed as follows:
$$(1 + S_3)^3 = \frac{(1 + S_2)^2}{(1 + 2y1y)}$$

$$(1 + 2y1y) = \frac{(1 + S_3)^3}{(1 + S_2)^2}$$

$$(1 + 2y1y) = \frac{(1.0611)^3}{(1.0576)^2} = 1.0681 $$

$$= 2y1y = 6.81\%$$
\
    (Module 57.1, LOS 57.a)

In [13]:
# Question 4 of 6
# Given that the 2-year spot rate is 5.76% and 
# the 3-year spot rate is 6.11%, what is the 1-year forward rate 
# starting two years from now?


# EXAMPLE: COMPUTING A FORWARD RATE FROM SPOT RATES
S_2y1y = (1.0611**3) / (1.0576**2) - 1
print(round(S_2y1y,4)*100)

6.81


**QUESTION 5/6:**

Given that the one-year spot rate is 6.05% and the two-year spot rate is 7.32%, assuming annual compounding what is the one-year forward rate starting one year from now?

* **ANSWER:**  \
    8.61\%
* **EXPLANATION:** \
    The forward rate is computed as follows:
$$
\text{forward-rate}_{1,2} = \frac{\text{1 + spot rate 0,2}^2}{\text{1 + spot rate 0,1}^1} - 1 = \frac{(1+0.0732)^2}{(1+0.0605)^1}
-1 = 8.61\%
$$

    \
    (Module 57.1, LOS 57.b)

In [14]:
# Question 5 of 6

# EXAMPLE: COMPUTING A FORWARD RATE FROM SPOT RATES
fwd_1y1y = (1.0732)**2 / 1.0605 - 1
print(round(fwd_1y1y,4)*100)

8.61


**QUESTION 6/6:**

A 2-year option-free bond (par value of //$10,000) has an annual coupon of 15\%. An investor determines that the spot rate of year 1 is 16\% and the year 2 spot rate is 17\%. The bond price is closest to:

* **ANSWER:**  \
    \\$9,694.
* **EXPLANATION:** \
    We can calculate the price of the bond by discounting each of the annual payments by the appropriate spot rate and finding the sum of the present values. 
    
$$
\text{Price} = \frac{1,500}{(1.16)^1} + \frac{11,500}{(1.17)^2} = \$9,694
$$
    \
    (Module 57.1, LOS 57.a)


In [15]:
# Question 6 of 6

# Calculate the coupon
print(10000 * 0.15)
# Calculate the coupon + principal
print(1500+10000)
print("--------")

# EXAMPLE: VALUING A BOND USING SPOT RATES
Q6 = (1500 / (1.16)**1) + (11500 / (1.17)**2)
print(round(Q6,2))

1500.0
11500
--------
9694.01


<hr>

**Question 6/6 Alternative solution**

Or, in keeping with the notion that each cash flow is a separate bond, sum the following transactions on your financial calculator:

        N=1, I/Y=16.0, PMT=0, FV=1,500, CPT PV=1,293

        N=2, I/Y=17.0, PMT=0, FV=11,500, CPT PV=8,401

        Price = 1,293 + 8,401 = 9,694

In [16]:
# Question 6/6 Alternative solution

# Use NumpyFinancial to solve PV 1
PV1_2 = npf.pv(nper=1, pmt=0, rate=0.16, fv=1500)
print(round(PV1_2,2))

# Use NumpyFinancial to solve PV 2
PV2_2 = npf.pv(nper=2, pmt=0, rate=0.17, fv=11500)
print(round(PV2_2,2))

price = PV1_2*-1 + PV2_2*-1
print(round(price,2))

-1293.1
-8400.91
9694.01
